## 2-1. 데이터 읽기

In [1]:
import numpy as np
import pandas as pd


# u.user 파일을 DataFrame으로 읽기 
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('../Data/u.user', sep='|', names=u_cols, encoding='latin-1')
users = users.set_index('user_id')
users.head()

,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [2]:
# u.item 파일을 DataFrame으로 읽기
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 
        'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 
        'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 
        'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('../Data/u.item', sep='|', 
                    names=i_cols, encoding='latin-1') # names: dataframe의 열 이름으로 할당
# 'movie_id'라는 열을 DataFrame의 인덱스로 설정
movies = movies.set_index('movie_id')   # movie_id 열을 인덱스로 설정
movies.head()

,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [5]:
# u.data 파일을 DataFrame으로 읽기 (rating 1~5점/timestap 연도/날짜/시간을 숫자로 변환한 값)
r_cols  = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../Data/u.data', sep='\t', names=r_cols, encoding='latin-1') # tab으로 분리
ratings = ratings.set_index('user_id')
ratings.head()

,movie_id,rating,timestamp
user_id,,,
196,242,3,881250949
186,302,3,891717742
22,377,1,878887116
244,51,2,880606923
166,346,1,886397596


## 2-2. 인기제품(best-seller) 방식

In [14]:
# Best-seller 추천 

def recom_movie1(n_items=5):
    # movie_mean을 내림차순으로 n_items 만큼 movie_sort에 저장
    movie_sort      = movie_mean.sort_values(ascending=False)[:n_items]
    # movies에서 movie_sort에 해당하는 정보를 recom_movie에 저장
    # df.loc[index(, column)], df.loc[[index_labels], [column_labels]]
    recom_movies    = movies.loc[movie_sort.index]
    # recom_movies에서 영화제목만 골라내어 저장
    recommendations = recom_movies['title']
    return recommendations

# 각 movie_id 별 rating의 평균(mean)을 계산하여 저장 (Series 형식)
movie_mean = ratings.groupby(['movie_id'])['rating'].mean()
print(recom_movie1(5))

# recom_movie1과 같은 역할. 코드를 한 줄로 만든 ver.
def recom_movie2(n_items):
    return movies.loc[movie_mean.sort_values(ascending=False)[:n_items].index]['title']

print(recom_movie2(5))

movie_id
814                         Great Day in Harlem, A (1994)
1599                        Someone Else's America (1995)
1201           Marlene Dietrich: Shadow and Light (1996) 
1122                       They Made Me a Criminal (1939)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object
movie_id
814                         Great Day in Harlem, A (1994)
1599                        Someone Else's America (1995)
1201           Marlene Dietrich: Shadow and Light (1996) 
1122                       They Made Me a Criminal (1939)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object


## 2-3. 추천 시스템의 정확도 측정

In [18]:
# 정확도 계산 
def RMSE(y_true, y_pred):   
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

rmse = []
for user in set(ratings.index): # ratings에 있는 user_id를 다 추출해서 user마다 루프
    # 각 user가 평가한 영화의 평점을 y_true에 저장
    y_true = ratings.loc[user]['rating']    
    # 해당 user가 평가한 영화의 전체 평점 평균을 y_pred에 저장
    y_pred = movie_mean[ratings.loc[user]['movie_id']]  
    # 해당 user의 rmse 계산 (예측값(전체유저의 평점평균), 실제값 (해당유저의 평점))
    accuracy = RMSE(y_true, y_pred)
    # 현재 사용자의 rmse(정확도)를 리스트에 추가
    rmse.append(accuracy)

# 전체 사용자의 rmse의 평균
print(np.mean(rmse))

0.996007224010567
